In [ ]:
!pip install Sastrawi
!pip install fuzzywuzzy
!pip install tensorflow

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from fuzzywuzzy import fuzz
from nltk.tokenize import word_tokenize
from pathlib import Path
from zipfile import ZipFile

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, Flatten, Concatenate, Input

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tv = TfidfVectorizer(max_features=5000)
stem = StemmerFactory().create_stemmer()
stopword = StopWordRemoverFactory().create_stop_word_remover()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 17.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def preprocessing(data):
    data = data.lower()
    data = stem.stem(data)
    data = stopword.remove(data)
    return data

In [ ]:
ratings = pd.read_csv('/content/drive/MyDrive/Dataset - Rating.csv')
# Membuat daftar semua user_id dan place_id yang mungkin ada dalam dataset
all_user_ids = ratings['User_Id'].unique()
all_place_ids = ratings['Place_Id'].unique()

# # Membuat DataFrame kosong untuk menampung baris baru
new_rows = pd.DataFrame(columns=['User_Id', 'Place_Id', 'Place_Rating'])

# # Iterasi melalui setiap kombinasi user_id dan place_id
for user_id in all_user_ids:
    for place_id in all_place_ids:
        # Memeriksa apakah kombinasi sudah ada dalam dataset rating
        if not ((ratings['User_Id'] == user_id) & (ratings['Place_Id'] == place_id)).any():
            # Jika kombinasi belum ada, tambahkan baris baru dengan nilai default 0
            new_row = {'User_Id': user_id, 'Place_Id': place_id, 'Place_Rating': np.nan}
            new_rows = new_rows.append(new_row, ignore_index=True)

# # Menambahkan baris baru ke dataset rating
updated_rating_data = pd.concat([ratings, new_rows], ignore_index=True)
print(updated_rating_data)

Streaming output truncated to the last 5000 lines.
<ipython-input-5-82f9e181bce6>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_rows = new_rows.append(new_row, ignore_index=True)
<ipython-input-5-82f9e181bce6>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_rows = new_rows.append(new_row, ignore_index=True)
<ipython-input-5-82f9e181bce6>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_rows = new_rows.append(new_row, ignore_index=True)
<ipython-input-5-82f9e181bce6>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_rows = new_rows.append(new_row, ignore_index=True)
<ipython-input-5-82f9e181bce6>:16: FutureWarning:

      User_Id  Place_Id            Nama Tempat  Place_Rating
0         1.0       2.0   Agung Bali Oleh-Oleh           3.0
1         1.0       7.0       Aloha Ubud Swing           5.0
2         1.0       8.0         Bali Bird Park           5.0
3         1.0      12.0      Big Garden Corner           2.0
4         1.0      13.0  Wanagiri Hidden Hills           5.0
...       ...       ...                    ...           ...
7496    100.0      28.0                    NaN           NaN
7497    100.0      66.0                    NaN           NaN
7498    100.0      14.0                    NaN           NaN
7499    100.0      61.0                    NaN           NaN
7500    100.0      62.0                    NaN           NaN

[7501 rows x 4 columns]


<ipython-input-5-82f9e181bce6>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_rows = new_rows.append(new_row, ignore_index=True)
<ipython-input-5-82f9e181bce6>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_rows = new_rows.append(new_row, ignore_index=True)
<ipython-input-5-82f9e181bce6>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_rows = new_rows.append(new_row, ignore_index=True)
<ipython-input-5-82f9e181bce6>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_rows = new_rows.append(new_row, ignore_index=True)
<ipython-input-5-82f9e181bce6>:16: FutureWarning: The frame.append method is deprecated and will be 

In [ ]:
# Ambil data rating yang telah diperbarui
updated_rating_data = updated_rating_data.copy()

# Buat dictionary mapping antara place_id dan place_name dari dataset place
place_mapping = ratings.set_index('Place_Id')['Nama Tempat'].to_dict()

# Mengisi nilai NaN di kolom 'Nama Tempat' berdasarkan 'Place_Id' menggunakan metode fillna() dan mapping dari dataset place
updated_rating_data['Nama Tempat'] = updated_rating_data['Nama Tempat'].fillna(updated_rating_data['Place_Id'].map(place_mapping))

In [ ]:
updated_rating_data.sort_values(by = 'Place_Id')

,User_Id,Place_Id,Nama Tempat,Place_Rating
1620,55.0,1.0,Satria Agrowisata,4.0
7259,95.0,1.0,Satria Agrowisata,NaN
2220,75.0,1.0,Satria Agrowisata,1.0
390,14.0,1.0,Satria Agrowisata,4.0
4785,40.0,1.0,Satria Agrowisata,NaN
...,...,...,...,...
5336,52.0,75.0,Wisata Agro Teba Sari Bali,NaN
4840,41.0,75.0,Wisata Agro Teba Sari Bali,NaN
479,16.0,75.0,Wisata Agro Teba Sari Bali,5.0
3624,14.0,75.0,Wisata Agro Teba Sari Bali,NaN


In [ ]:
# Membuat matriks rating dari updated_rating_data
user_ids = updated_rating_data['User_Id']
item_ids = updated_rating_data['Place_Id']
ratings = updated_rating_data['Place_Rating']

num_users = int(max(user_ids) + 1)
num_items = int(max(item_ids) + 1)

# Membuat matriks rating dengan ukuran num_users, num_items yang isinya adalah rating
rating_matrix = np.zeros((num_users, num_items))
for user_id, item_id, rating in zip(user_ids, item_ids, ratings):
    user_id = int(user_id)
    item_id = int(item_id)
    rating_matrix[user_id, item_id] = rating


In [ ]:
# Menggunakan SimpleImputer untuk mengisi nilai rating yang hilang
imputer = SimpleImputer(strategy='mean')
imputed_rating_matrix = imputer.fit_transform(rating_matrix)

# Membuat array indeks bilangan bulat dari user_ids dan item_ids
user_indices = user_ids.astype(int)
item_indices = item_ids.astype(int)

# Menyimpan hasil imputasi ke dalam updated_rating_data
updated_rating_data['Place_Rating'] = imputed_rating_matrix[user_indices, item_indices]

In [ ]:
# Baca data turisme
tourism_data = pd.read_csv('/content/drive/MyDrive/datasetRevisi.csv', encoding='latin-1')
# Baca data rating
rating_data = updated_rating_data.copy()

# Gabungkan data turisme dengan data rating berdasarkan kolom 'Place_Id'
merged_data = pd.merge(tourism_data, rating_data, on='Place_Id', how='left')
merged_data

In [ ]:
merged_data.Description = merged_data.Description.apply(preprocessing)
merged_data.drop(['Gambar','Coordinate','Lat', 'Long','Unnamed: 13'],axis=1,inplace=True)

In [ ]:
# Membuat dictionary mapping antara place_id dan place_name
place_mapping = dict(zip(merged_data['Place_Id'], merged_data['Place_Name']))

# Mengkonversi tipe data kolom 'User_Id' dan 'Place_Id' menjadi float32
merged_data['User_Id'] = merged_data['User_Id'].values.astype(np.float32)
merged_data['Place_Id'] = merged_data['Place_Id'].values.astype(np.float32)

# Menghitung jumlah pengguna dan tempat unik
num_users = merged_data['User_Id'].nunique()
num_places = merged_data['Place_Id'].nunique()

# Membangun model content-based filtering
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(merged_data['Description'])
num_features = tfidf_matrix.shape[1]

cbf_input = Input(shape=(num_features,))
cbf_dense_1 = Dense(128, activation='relu')(cbf_input)
cbf_dense_2 = Dense(64, activation='relu')(cbf_dense_1)
cbf_output = Dense(1)(cbf_dense_2)

# Membangun model collaborative filtering
embedding_size = 64

user_input = Input(shape=(1,)) # menyediakan input dalam bentuk 1 dimensi
user_embedding = Embedding(num_users + 1, embedding_size)(user_input)
user_flatten = Flatten()(user_embedding)

place_input = Input(shape=(1,))
place_embedding = Embedding(num_places + 1, embedding_size)(place_input)
place_flatten = Flatten()(place_embedding)

cf_concatenated = Concatenate()([user_flatten, place_flatten])
cf_dense_1 = Dense(256, activation='relu')(cf_concatenated)
cf_dense_2 = Dense(128, activation='relu')(cf_dense_1)
cf_dense_3 = Dense(64, activation='relu')(cf_dense_2)
cf_output = Dense(1)(cf_dense_3)

# Menggabungkan model content-based filtering dan collaborative filtering
merged_model = Concatenate()([cbf_output, cf_output])
final_output = Dense(1)(merged_model)

hybrid_model = Model(inputs=[cbf_input, user_input, place_input], outputs=final_output)

# Mengompilasi dan melatih model
hybrid_model.compile(optimizer='adam', loss='mean_squared_error')

# Perform the model training with EarlyStopping
hybrid_model.fit([tfidf_matrix.toarray(), merged_data['User_Id'].values.reshape(-1), merged_data['Place_Id'].values.reshape(-1)],
                 merged_data['Place_Rating'], epochs=10, batch_size=64)

# Save the model to a file
hybrid_model.save('hybrid_model.h5')

Epoch 1/10
118/118 [==============================] - 3s 8ms/step - loss: 2.0101
Epoch 2/10
118/118 [==============================] - 1s 8ms/step - loss: 0.4471
Epoch 3/10
118/118 [==============================] - 1s 7ms/step - loss: 0.4363
Epoch 4/10
118/118 [==============================] - 1s 7ms/step - loss: 0.4263
Epoch 5/10
118/118 [==============================] - 1s 7ms/step - loss: 0.4130
Epoch 6/10
118/118 [==============================] - 1s 7ms/step - loss: 0.3876
Epoch 7/10
118/118 [==============================] - 1s 7ms/step - loss: 0.3632
Epoch 8/10
118/118 [==============================] - 1s 7ms/step - loss: 0.3426
Epoch 9/10
118/118 [==============================] - 1s 7ms/step - loss: 0.3129
Epoch 10/10
118/118 [==============================] - 1s 10ms/step - loss: 0.2877


In [ ]:
# Menghitung MSE pada data latih
mse = hybrid_model.evaluate([tfidf_matrix.toarray(), merged_data['User_Id'].values.reshape(-1), merged_data['Place_Id'].values.reshape(-1)],
                            merged_data['Place_Rating'])
print('MSE: {}'.format(mse))

235/235 [==============================] - 2s 5ms/step - loss: 0.2310
MSE: 0.23099692165851593


In [ ]:
user_id = updated_rating_data.User_Id.sample(1).iloc[0]
user_data = merged_data[merged_data['User_Id'] == user_id]
user_id

22.0

In [ ]:
place_ids = tourism['Place_Id'].values
user_ids = np.full(len(place_ids), user_id)
input_data = pd.DataFrame({'User_Id': user_ids, 'Place_Id': place_ids})

# Ubah ukuran tfidf_matrix sesuai dengan jumlah sampel input_data
tfidf_matrix = tfidf_matrix[:len(place_ids)]

In [ ]:
# Melakukan prediksi rating untuk tempat yang belum dikunjungi
predicted_ratings = hybrid_model.predict([tfidf_matrix.toarray(), input_data['User_Id'].values.reshape(-1), input_data['Place_Id'].values.reshape(-1)])
predicted_ratings = np.clip(predicted_ratings, 0, 5)  # Membatasi nilai dalam rentang [0, 5]
predicted_ratings = (predicted_ratings / np.max(predicted_ratings)) * 5  # Menskalakan nilai ke rentang [0, 5]

# Membuat DataFrame dengan kolom 'Place_Name' dan 'Predicted_Rating'
predictions_df = pd.DataFrame({'Place_Id' :input_data['Place_Id'], 'Place_Name': input_data['Place_Id'].map(place_mapping), 'Predicted_Rating': predicted_ratings.flatten()})

# Mengurutkan predictions_df berdasarkan 'Predicted_Rating' secara menurun
predictions_df = predictions_df.sort_values(by='Predicted_Rating', ascending=False)
predictions_df

rating = pd.read_csv('/content/drive/MyDrive/ratingDataset.csv')
places_visited = rating[rating['User_Id'] == user_id]['Place_Id']

recommend_place = predictions_df[~predictions_df['Place_Id'].isin(places_visited)].head(10)

2/2 [==============================] - 0s 7ms/step
                      Place_Name  Predicted_Rating
54        Teras Sawah Tegalalang          5.000000
71         Danau Beratan Bedugul          4.970796
21       Air Terjun Tukad Cepung          4.936574
43      Galeri Seni Semar Kuning          4.884063
13               Pantai Seminyak          4.851740
52  Pusat Kesenian Ukir Desa Mas          4.813184
10               Pantai Nusa Dua          4.788850
50                Pura Saraswati          4.757193
26                    Pura Kehen          4.669592
41           Campuhan Ridge Walk          4.653401


In [ ]:
min_rating = np.min(rating_data['Place_Rating'])
max_rating = np.max(rating_data['Place_Rating'])
print("Minimum Rating:", min_rating)
print("Maximum Rating:", max_rating)

Minimum Rating: 1.0
Maximum Rating: 5.0


In [ ]:
model = tf.keras.models.load_model('hybrid_model.h5')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [ ]:
with open('hybrid_model.tflite', 'wb') as f:
    f.write(tflite_model)
